In [195]:
import numpy as np
import pandas as pd
import os
import glob
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 500
np.random.seed(231)
from sklearn.model_selection import train_test_split
import cv2


In [189]:
train_dir = "/Users/rugezhao/Desktop/imet/train"
data_dir = "/Users/rugezhao/Desktop/imet"
output_dir = "/Users/rugezhao/Desktop/GIT/CS231N_CZT/data"

In [46]:
train_df = pd.read_csv(data_dir+"/train.csv")
labels_df = pd.read_csv(data_dir+"/labels.csv")


In [48]:
train_attr_ohot = np.zeros((len(train_df), len(labels_df)), dtype=int)
for idx, attr_arr in enumerate(train_df.attribute_ids.str.split(" ").apply(lambda l: list(map(int, l))).values):
    print(idx,attr_arr)
    train_attr_ohot[idx, attr_arr] = 1

0 [147, 616, 813]
1 [51, 616, 734, 813]
2 [776]
3 [51, 671, 698, 813, 1092]
4 [13, 404, 492, 903, 1093]
5 [189, 279, 774, 800, 1051]
6 [188, 1034]
7 [1010, 1053]
8 [189, 541, 542, 993]
9 [70, 776, 794, 813, 1046, 1092]
10 [79, 1062]
11 [188, 668, 754]
12 [51, 675]
13 [147, 716, 903, 1092]
14 [161, 489, 704, 1100]
15 [188, 535]
16 [51, 212, 426, 586, 940]
17 [13, 616]
18 [737, 1009, 1046]
19 [25, 161, 784, 1059]
20 [189, 542, 670, 813, 949, 1092]
21 [147, 189, 418, 742, 1072, 1092]
22 [194, 480, 483, 485, 813, 1099]
23 [13, 813, 896]
24 [156, 1084]
25 [103, 180, 573]
26 [147, 1039]
27 [25, 161, 584, 616, 627, 738, 868]
28 [51, 212, 1059]
29 [835, 1059]
30 [51, 538, 747, 813, 1012, 1092]
31 [13, 813]
32 [156, 543, 753, 961, 1072]
33 [13, 552]
34 [13, 1013]
35 [156, 228, 1054]
36 [156, 274, 538, 663, 1059]
37 [51, 339, 492, 961]
38 [13, 501, 670, 871]
39 [13, 857]
40 [194, 616, 844]
41 [13, 668]
42 [121, 464]
43 [189, 439, 733, 813]
44 [615, 813, 896]
45 [156, 466, 616, 739, 813]
46 [147,

938 [813, 896]
939 [51, 833]
940 [79, 539, 737, 776, 1046]
941 [70, 628, 813, 833, 1020, 1092]
942 [189, 436]
943 [156, 485, 603, 780]
944 [501, 580, 871, 993, 1009]
945 [51, 76, 650, 671, 780]
946 [13, 405, 896, 1092]
947 [79, 1039]
948 [51, 124, 339, 734, 1059]
949 [13, 670, 825]
950 [96, 236, 962]
951 [147, 510, 764, 813]
952 [304, 487, 607]
953 [194, 404]
954 [194, 480, 844]
955 [79, 487, 624]
956 [188, 671, 1059]
957 [135, 554, 954, 369]
958 [13, 675]
959 [99, 783]
960 [477, 671, 1035]
961 [51, 212, 477, 489, 554, 991]
962 [26, 156, 436]
963 [194, 950]
964 [354, 421]
965 [738, 988]
966 [189, 583, 633, 761]
967 [121]
968 [147, 716]
969 [99, 304, 434, 744]
970 [25, 161, 744, 847, 1059]
971 [51, 393, 612, 671]
972 [189, 1039]
973 [158, 1085]
974 [194, 497, 498, 823, 970]
975 [51, 615, 633, 835]
976 [51, 627, 813, 993, 1092]
977 [99, 962]
978 [147, 889, 1092, 1093]
979 [14, 716]
980 [13, 538, 690, 813, 1092]
981 [147, 733, 813, 1046]
982 [189, 252, 813]
983 [194, 600, 809, 844]
984 [3

1550 [51, 480, 483, 670, 813, 1092]
1551 [51, 718, 858]
1552 [676, 1046]
1553 [121, 433]
1554 [189, 1034]
1555 [189, 378, 647]
1556 [188, 671, 1034]
1557 [188, 922]
1558 [51, 554, 744, 896]
1559 [18, 161, 331, 758, 903, 1092]
1560 [13, 559]
1561 [13, 405, 896, 1092]
1562 [335, 450, 718]
1563 [813, 896]
1564 [189, 1034]
1565 [121, 822]
1566 [70, 501, 545, 548, 776]
1567 [147, 956]
1568 [121, 488, 534, 598, 734, 739, 813]
1569 [78, 586, 822]
1570 [13, 903]
1571 [189, 671, 768, 780, 852]
1572 [1062]
1573 [189, 813, 1092]
1574 [147, 671]
1575 [13, 626]
1576 [147, 698, 1092]
1577 [121, 1034]
1578 [354, 497, 498]
1579 [1092]
1580 [79, 961, 1092]
1581 [156, 216, 867]
1582 [477, 671, 369]
1583 [51, 450]
1584 [29, 744, 813, 903, 1092]
1585 [188, 884]
1586 [79, 737, 776, 1046]
1587 [51, 747, 813, 916, 1092]
1588 [194, 671, 972, 1099]
1589 [111, 483, 738, 771, 776, 830]
1590 [121, 1098]
1591 [582, 1023]
1592 [147, 716, 1092]
1593 [147, 552]
1594 [13, 665, 813, 830, 1046]
1595 [51, 744, 813, 1092]

2299 [147, 477, 734, 800, 1092]
2300 [202, 481, 671, 780, 1059]
2301 [111, 858, 908]
2302 [147, 283, 813, 1025, 1092]
2303 [188, 477, 671, 1034]
2304 [51, 483, 554, 896, 975, 1025]
2305 [13, 790, 813, 896]
2306 [1084, 369]
2307 [156, 358, 487, 744, 813]
2308 [56, 497, 498]
2309 [147, 229, 492, 616]
2310 [79, 925]
2311 [1092]
2312 [121, 1059]
2313 [105, 111, 477, 612, 671, 780]
2314 [13, 698, 833, 896]
2315 [51, 738]
2316 [51, 580, 833]
2317 [79, 612, 624]
2318 [147, 413, 430, 439, 718, 858]
2319 [259, 670, 718, 813]
2320 [51, 666]
2321 [99, 304, 433, 1099]
2322 [13, 480, 483, 776]
2323 [13, 405, 896, 1092]
2324 [189, 738, 863]
2325 [127, 884]
2326 [121, 1059]
2327 [147]
2328 [259, 663]
2329 [51, 615, 678]
2330 [189, 480, 813, 1033, 1092]
2331 [282, 1059]
2332 [79, 734, 1046]
2333 [147, 833]
2334 [194, 1034]
2335 [438, 369]
2336 [79, 962]
2337 [147, 517, 780]
2338 [145, 579]
2339 [496]
2340 [13, 746, 813, 903]
2341 [147, 1092]
2342 [147, 550]
2343 [13, 978]
2344 [489, 369]
2345 [147, 53

3177 [70, 543, 579, 946]
3178 [563, 1061, 369]
3179 [833]
3180 [98, 489]
3181 [23, 708, 962]
3182 [13, 405, 903, 1092]
3183 [121, 1059]
3184 [79, 653]
3185 [51, 580]
3186 [189, 552, 554, 744]
3187 [415]
3188 [332, 598, 616, 739, 994]
3189 [439]
3190 [231, 1039]
3191 [147, 733, 813]
3192 [51, 583, 663, 800]
3193 [18, 161, 331, 738, 1059]
3194 [345, 420, 962]
3195 [671, 1059, 1099]
3196 [13, 1092]
3197 [86, 1032]
3198 [1039, 369]
3199 [332, 1005]
3200 [1059]
3201 [79, 489]
3202 [51, 393, 1030]
3203 [51, 813, 895, 896]
3204 [428, 647]
3205 [615, 369]
3206 [147, 742, 927, 1092]
3207 [13, 716]
3208 [189, 744, 903, 1092]
3209 [147, 1034]
3210 [51, 813, 827, 1092]
3211 [184, 420]
3212 [559]
3213 [194, 783, 968, 1092]
3214 [236, 1039]
3215 [79, 612]
3216 [147, 813]
3217 [189, 626, 780]
3218 [188, 428, 1097]
3219 [79, 498, 1023]
3220 [79, 584, 813, 971]
3221 [121, 433]
3222 [188, 428, 1085]
3223 [13, 405, 896, 1092]
3224 [51, 552, 671, 780]
3225 [51, 520, 744]
3226 [51, 212, 1059]
3227 [194, 10

...

104771 [156, 501, 548, 650, 744]
104772 [147, 655, 671, 738, 1046]
104773 [51, 420, 733, 813]
104774 [111, 813, 835, 931, 1032, 1092]
104775 [598, 1050]
104776 [185, 552]
104777 [156, 418, 808]
104778 [147, 713]
104779 [579, 581, 753]
104780 [879]
104781 [51, 212, 513]
104782 [111, 490]
104783 [813]
104784 [26, 156, 720]
104785 [147, 976]
104786 [671, 886, 894]
104787 [420, 737, 833, 834, 1092]
104788 [155, 189, 671, 768]
104789 [308, 671, 1034]
104790 [33, 582, 1023]
104791 [663]
104792 [189, 615]
104793 [188, 612, 671]
104794 [188, 1025, 1034]
104795 [184, 884]
104796 [79, 555]
104797 [189, 420]
104798 [194, 671, 1092]
104799 [1046]
104800 [51, 492, 524, 698, 813, 1092]
104801 [13, 866, 1046]
104802 [189, 485]
104803 [280, 754]
104804 [188, 496]
104805 [147, 813, 952, 1092]
104806 [156, 485, 542, 579, 916, 939]
104807 [331, 584]
104808 [360, 1059]
104809 [563, 1061, 369]
104810 [259, 485, 607, 1101]
104811 [420, 813, 369]
104812 [45, 972]
104813 [161, 864]
104814 [189, 1034]
104815 [

105771 [189, 768]
105772 [189, 616, 734, 813, 833]
105773 [156, 535, 661, 813, 835, 887]
105774 [147, 813, 949, 1072]
105775 [127, 922]
105776 [127, 597]
105777 [13, 420, 813, 1033]
105778 [671, 780, 1059]
105779 [133, 189, 418, 543, 579, 833, 1072]
105780 [147, 1092]
105781 [858]
105782 [189, 552]
105783 [147, 626]
105784 [79, 489, 1024]
105785 [259, 590, 808, 813, 833, 1092]
105786 [111, 738]
105787 [147, 485, 681, 1092, 1099]
105788 [813, 949]
105789 [58, 738, 1034]
105790 [13, 896]
105791 [121, 754]
105792 [1084, 369]
105793 [121, 464]
105794 [194, 844, 1092]
105795 [13, 813]
105796 [923, 975, 1046]
105797 [189, 306, 813, 895, 896]
105798 [189, 418, 434, 580, 835, 1072]
105799 [13, 813, 896]
105800 [194, 671, 1092]
105801 [121, 718, 813]
105802 [147, 559, 622, 835]
105803 [13, 552]
105804 [180, 733, 813, 1084]
105805 [17, 131, 671, 768]
105806 [147, 744, 813, 903]
105807 [194, 576]
105808 [121, 433, 724]
105809 [189, 1034]
105810 [189, 434]
105811 [147, 307, 420, 663, 813, 1046, 10

106770 [121, 704, 791]
106771 [13, 567]
106772 [121, 1061]
106773 [147, 813]
106774 [156, 485, 835]
106775 [13, 519]
106776 [13, 896, 1092]
106777 [188, 558, 744]
106778 [70, 671, 1058]
106779 [188, 1034]
106780 [194, 1098]
106781 [189, 813]
106782 [147, 738, 1068]
106783 [1034, 369]
106784 [156, 541, 543, 581, 813, 1092]
106785 [121, 1098]
106786 [188, 758]
106787 [189, 480, 483, 501, 613, 738]
106788 [194, 615, 975, 1099]
106789 [161, 204, 813, 974]
106790 [156, 813, 834, 1071]
106791 [194, 925]
106792 [51, 603, 671]
106793 [125, 955]
106794 [259, 579, 737, 776, 939, 1046]
106795 [156, 701, 776, 1092]
106796 [51, 813, 950, 1046, 1092]
106797 [155, 189, 671, 768]
106798 [147, 485]
106799 [99, 784]
106800 [1059]
106801 [194, 1034]
106802 [194, 629, 671, 886, 1099]
106803 [147, 813, 896]
106804 [483, 494, 913, 931]
106805 [156, 485, 603]
106806 [189, 768, 956]
106807 [79, 1059]
106808 [464, 634]
106809 [194, 438]
106810 [194, 1034]
106811 [316, 813, 1020]
106812 [335, 538, 813, 1092]
10

107770 [25, 161, 584]
107771 [79, 462, 1092]
107772 [189, 949]
107773 [51, 628]
107774 [420, 813]
107775 [51, 1059]
107776 [178, 661, 1053]
107777 [239, 809]
107778 [156, 406, 645, 912, 967, 1046]
107779 [13, 405, 896, 1092]
107780 [189, 485, 671, 691, 1055, 1096]
107781 [194, 671]
107782 [147, 978]
107783 [189, 418, 420, 579, 580, 946]
107784 [1092]
107785 [189, 449, 705, 734]
107786 [182, 744, 922]
107787 [51, 1059]
107788 [51, 813]
107789 [304, 640, 718, 896]
107790 [51, 450, 700]
107791 [563, 369]
107792 [51, 552]
107793 [194, 844]
107794 [156, 671, 1059]
107795 [70, 442, 542]
107796 [111, 813, 966]
107797 [194, 404, 738, 844]
107798 [25, 161, 584, 784, 813, 974, 994]
107799 [116, 161, 487, 1101]
107800 [194, 489, 780]
107801 [156, 420, 967, 1092]
107802 [188, 671, 813]
107803 [480, 483, 519, 738]
107804 [194, 1034, 1059]
107805 [147, 626, 671, 780]
107806 [13, 1056]
107807 [51, 813, 903]
107808 [13, 519, 696, 1094]
107809 [418, 579, 747, 753, 935]
107810 [202, 1059]
107811 [259, 4

108770 [156, 485, 671]
108771 [147, 813, 966]
108772 [147, 501, 559, 649, 871, 1092]
108773 [875, 369]
108774 [156, 418, 541, 813]
108775 [628]
108776 [51, 492, 890, 1046]
108777 [121, 617]
108778 [189, 517, 718]
108779 [147, 251, 451, 519, 616, 733, 813, 1092]
108780 [116, 161, 487]
108781 [51, 124, 339, 813, 903]
108782 [125, 1062]
108783 [489]
108784 [188, 428]
108785 [99, 758]
108786 [188, 597]
108787 [121, 573, 715]
108788 [615, 813, 896]
108789 [111, 494, 973, 1046]
108790 [705, 869, 1003]
108791 [718, 813, 903]
108792 [774]
108793 [121, 681, 961]
108794 [13, 626]
108795 [64, 189, 744, 835, 910]
108796 [189, 519, 738, 760, 860, 870]
108797 [175, 586, 738]
108798 [147, 434, 616, 813, 1092]
108799 [479, 598, 739, 1037]
108800 [189, 631]
108801 [147, 522, 830]
108802 [70, 753, 813, 1032, 1072, 1092]
108803 [487, 369]
108804 [813]
108805 [121, 1059]
108806 [1, 1039]
108807 [335, 430, 597, 882]
108808 [147, 813]
108809 [121, 962]
108810 [147, 671, 780, 1034]
108811 [147, 671, 680]
108

In [50]:
train_attr_ohot

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [4]:
train_id_attribute_id = train_df.copy()
train_id_attribute_id_ohe = train_id_attribute_id.attribute_ids.str.split(' ', expand=True).stack().str.get_dummies().sum(level=0)

In [5]:
train_id_attribute_id_ohe = train_id_attribute_id[:5000].attribute_ids.str.split(' ', expand=True).stack().str.get_dummies().sum(level=0)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratios[2], random_state=231,stratify=y)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=ratios[1], random_state=231,stratify=y_train)

1         6
100       1
1000      4
1001      5
1002      4
1003      1
1004      1
1005      6
1006      1
1007      2
1008      3
1009     10
1010      3
1012      8
1013      4
1014      3
1015      3
1016      2
1018      3
1019     13
102       4
1020     39
1022     12
1023     19
1024      7
1025      6
1026      2
1027      1
1028      1
1029      1
103       1
1030      9
1032     11
1033      9
1034    143
1035     36
1036      3
1037      5
1038     10
1039     95
1040      8
1041      6
1042      2
1043      1
1044      1
1045      9
1046    237
1047      3
1048      1
1049      4
105       6
1050      1
1051      7
1052      1
1053      1
1054      1
1055      5
1056      9
1057      2
1058      6
1059    284
106       2
1061     51
1062     76
1063      3
1064      8
1065      5
1067      2
1068      6
1069      3
107       4
1070      1
1071      6
1072     49
1073      2
1074      4
1075      1
1078      3
1080      2
1081      9
1082      2
1083      4
1084     47
1085

In [29]:
train_attr_ohot = np.zeros((len(train_df), len(labels_df)), dtype=int)
for idx, attr_arr in enumerate(train_df.attribute_ids.str.split(" ").apply(lambda l: list(map(int, l))).values):
    train_attr_ohot[idx, attr_arr] = 1


In [61]:
col_sum=pd.DataFrame(np.sum(train_id_attribute_id_ohe,axis=0))
col_sum=col_sum.reset_index()
infreq_labels = list(col_sum[col_sum.iloc[:,1] < min_count].iloc[:,0])
infreq_labels = [str(x) for x in infreq_labels]

In [93]:
train_id_attribute_id_ohe = pd.DataFrame(train_id_attribute_id_ohe,columns=labels_df.attribute_id)
col_sum=pd.DataFrame(np.sum(train_id_attribute_id_ohe,axis=0))
col_sum=col_sum.reset_index()
infreq_labels = list(col_sum[col_sum.iloc[:,1] < min_count].iloc[:,0])
infreq_labels = [str(x) for x in infreq_labels]

In [139]:
df_onehot.shape

(109237, 1104)

In [138]:
df_onehot[~df_onehot.id.isin(X_infreq)].shape

(109200, 1104)

The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2. To stratify sample the class needs at least 3 samples. So I decided to drop all samples that are tagged to a label that appeared less than 3 times.

# Data Split

In [143]:
def train_val_test_split(train_id_attribute_id, ratios = (0.8,0.1,0.1),min_count=3):
    train_id_attribute_id_ohe= np.zeros((len(train_df), len(labels_df)), dtype=int)
    for idx, attr_arr in enumerate(train_df.attribute_ids.str.split(" ").apply(lambda l: list(map(int, l))).values):
        train_id_attribute_id_ohe[idx, attr_arr] = 1
    train_id_attribute_id_ohe = pd.DataFrame(train_id_attribute_id_ohe,columns=labels_df.attribute_id)
    print("one hot encoding done")
    
    col_sum=pd.DataFrame(np.sum(train_id_attribute_id_ohe,axis=0))
    col_sum=col_sum.reset_index()
    infreq_labels = list(col_sum[col_sum.iloc[:,1] < min_count].iloc[:,0])
    print("{} infrequent labels less than {} times found".format(len(infreq_labels),min_count))
    
    # drop the examples with infreq labels, i.e. if 1 in any of these columns -->drop
    df_onehot = pd.concat([train_id_attribute_id,train_id_attribute_id_ohe], axis=1)
    df_onehot = df_onehot.drop(columns=["attribute_ids"])
    X_infreq = list(df_onehot.loc[(infreq_cols!=0).any(axis=1)].id)
    
    df_onehot = df_onehot[~df_onehot.id.isin(X_infreq)]
     
    X = df_onehot.id
    y = df_onehot.iloc[:,1:]
    # use stratify=y for stratified sampling -->> impossible!! since they need to treat combination of label as a class
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratios[2], random_state=231,stratify=None)
    print("test set done")
    # use stratify =y_train for stratified sampling -->impossible!!
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=ratios[1], random_state=231,stratify=None)
    print("val set done")
    print("train set done")
    return X_train, X_val, X_test, X_infreq

In [171]:
np.random.seed(231)
X_train, X_val, X_test, X_infreq = train_val_test_split(train_id_attribute_id)

one hot encoding done
26 infrequent labels less than 3 times found
test set done
val set done
train set done


## Check distribution

In [172]:
df_one_train = df_onehot[df_onehot.id.isin(X_train)]
df_one_val = df_onehot[df_onehot.id.isin(X_val)]
df_one_test = df_onehot[df_onehot.id.isin(X_test)]

In [173]:
col_pct_train=pd.DataFrame(np.sum(df_one_train.iloc[:,1:],axis=0))/df_one_train.shape[0]
col_pct_val=pd.DataFrame(np.sum(df_one_val.iloc[:,1:],axis=0))/df_one_val.shape[0]
col_pct_test=pd.DataFrame(np.sum(df_one_test.iloc[:,1:],axis=0))/df_one_test.shape[0]

In [174]:
col_pct_train.sort_values(0,ascending=False)

,0
813,0.183829
1092,0.130805
147,0.124440
189,0.094695
13,0.083141
671,0.077681
51,0.070298
194,0.068195
1059,0.059818
121,0.059501


In [175]:
col_pct_val.sort_values(0,ascending=False)

,0
813,0.185083
1092,0.128205
147,0.119760
189,0.092796
13,0.088523
671,0.075092
51,0.067766
194,0.063797
1059,0.059320
121,0.058506


In [176]:
col_pct_test.sort_values(0,ascending=False)

,0
813,0.172711
1092,0.132418
147,0.122253
189,0.099451
13,0.084890
671,0.073626
194,0.067308
51,0.066575
121,0.064469
1059,0.062912


In [180]:
# export split to csv
train_df['train'] = 0
train_df['val']=0
train_df['test']=0


,id,attribute_ids,train,val,test
0,1000483014d91860,147 616 813,0,0,0
1,1000fe2e667721fe,51 616 734 813,0,0,0
2,1001614cb89646ee,776,0,0,0
3,10041eb49b297c08,51 671 698 813 1092,0,0,0
4,100501c227f8beea,13 404 492 903 1093,0,0,0
5,10050ed12fbad46d,189 279 774 800 1051,0,0,0
6,100543a032517972,188 1034,0,0,0
7,1006665c0aad488,1010 1053,0,0,0
8,1007057734dba6df,189 541 542 993,0,0,0
9,1008abd71f3ed5bc,70 776 794 813 1046 1092,0,0,0


In [191]:
train_df.loc[train_df.id.isin(list(X_train)),'train'] = 1
train_df.loc[train_df.id.isin(list(X_val)),'val'] = 1
train_df.loc[train_df.id.isin(list(X_test)),'test'] = 1
train_df.to_csv(output_dir+"/train_split.csv",index=False)

# TODO: Change Folder Structure - move .png according to train_split.csv

# Training preprocessing

In [ ]:
# pause for now since it can be done at run time using pytorch
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

# Val, Test Preprocessing

In [ ]:
# pause for now since it can be done at run time using pytorch
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

# Training Per Channel Mean and Std - after preprocessing

In [255]:
import os
train_files = os.listdir(train_dir)

In [261]:
sum_x_rgb = np.array([0.0]*3)
sum_x2_rgb = np.array([0.0]*3)
file_count =0
pixel_count=0
for i, file in enumerate(train_files):
    print("{}/{} files read".format(i+1, len(train_files)))
    file_count+=1
    
    image = cv2.imread(train_dir+"/"+file)
    pixel_count += image.shape[0]*image.shape[1]
    # R, G, B
    for i in range(3):
        sum_x_rgb[i] += np.sum(image[:,:,i].astype(float))
        sum_x2_rgb[i] += np.sum(image[:,:,i].astype(float)**2)
        
        

1/10 files read
2/10 files read
3/10 files read
4/10 files read
5/10 files read
6/10 files read
7/10 files read
8/10 files read
9/10 files read
10/10 files read


In [262]:
channel_mean = sum_x_rgb/pixel_count
channel_mean

array([135.49223492, 149.21130539, 161.92014048])

In [263]:
channel_sd = np.sqrt(sum_x2_rgb/pixel_count-channel_mean**2)
channel_sd

array([68.17929996, 68.57458389, 68.86966282])

# Generate small training sample for debugging 

sample by id from large training sample

In [192]:
sample_idx = np.random.choice(train_df.shape[0], 5000)
sample_train_df = train_df.iloc[sample_idx,:]